In [ ]:
# 기본 모듈 임포트
import os
import asyncio
from dotenv import load_dotenv

# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보 로드
load_dotenv()                   # true

In [ ]:
# 환경 변수 확인하기

# 마스킹 처리 함수 정의
def mask_key(key: str, visible_count: int = 2) -> str:
    if not key or len(key) <= visible_count:
        return '*' * len(key)
    return key[:visible_count] + '*' * (len(key) - visible_count)

# 환경변수 불러오기
api_key = os.getenv("GOOGLE_API_KEY")
if not api_key:
    raise ValueError("GOOGLE_API_KEY 환경 변수가 설정되지 않았습니다.")

# 마스킹된 형태로 출력
print(f"GOOGLE_API_KEY: {mask_key(api_key)}")           # GOOGLE_API_KEY: AI*************************************

In [ ]:
# LangSmith 추적 설정 (https://smith.langchain.com)

"""
- !pip install -qU langsmith
- !pip install -qU langchain-teddynote
    -> 제미나이와 poetry와의 의존성 충돌로 langchain_teddy 설치 X 
    -> langsmith로 진행
"""
# LangSmith 추적을 위한 라이브러리 임포트
from langsmith import traceable                                                             # @traceable 데코레이터 사용 시

# LangSmith 환경 변수 확인

print("\n--- LangSmith 환경 변수 확인 ---")
langchain_tracing_v2 = os.getenv('LANGCHAIN_TRACING_V2')
langchain_project = os.getenv('LANGCHAIN_PROJECT')
langchain_api_key_status = "설정됨" if os.getenv('LANGCHAIN_API_KEY') else "설정되지 않음"      # API 키 값은 직접 출력하지 않음

if langchain_tracing_v2 == "true" and os.getenv('LANGCHAIN_API_KEY') and langchain_project:
    print(f"✅ LangSmith 추적 활성화됨 (LANGCHAIN_TRACING_V2='{langchain_tracing_v2}')")
    print(f"✅ LangSmith 프로젝트: '{langchain_project}'")
    print(f"✅ LangSmith API Key: {langchain_api_key_status}")
    print("  -> 이제 LangSmith 대시보드에서 이 프로젝트를 확인해 보세요.")
else:
    print("❌ LangSmith 추적이 완전히 활성화되지 않았습니다. 다음을 확인하세요:")
    if langchain_tracing_v2 != "true":
        print(f"  - LANGCHAIN_TRACING_V2가 'true'로 설정되어 있지 않습니다 (현재: '{langchain_tracing_v2}').")
    if not os.getenv('LANGCHAIN_API_KEY'):
        print("  - LANGCHAIN_API_KEY가 설정되어 있지 않습니다.")
    if not langchain_project:
        print("  - LANGCHAIN_PROJECT가 설정되어 있지 않습니다.")

<small>

* --- LangSmith 환경 변수 확인 ---
* ✅ LangSmith 추적 활성화됨 (LANGCHAIN_TRACING_V2='true')
* ✅ LangSmith 프로젝트: 'LangChain-prantice'
* ✅ LangSmith API Key: 설정됨
*   -> 이제 LangSmith 대시보드에서 이 프로젝트를 확인해 보세요.

In [ ]:
# LangChain 및 Google GenAI 모델 관련 모듈 임포트
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI           # Google GenAI 임포트


print("\n--- LangChain 체인 설정 ---")                                # --- LangChain 체인 설정 ---

In [ ]:
# LLM 객체 정의하기

# model1 = gemini-1.5-flash
# model2 = gemini-2.5-falsh-lite

try:
    model1 = ChatGoogleGenerativeAI(                                      # 모델 호출
        model="gemini-1.5-flash",
        #temperature=0.1,
    )
    print("✅ gemini-1.5-flash 호출 성공.")
except Exception as e:                                                   # 디버깅 메시지
    print(f"❌ Google GenAI 모델 초기화 실패: {e}")
    print("  -> GEMINI_API_KEY 환경 변수가 올바르게 설정되었는지 확인하세요.")      # ✅ gemini-2.5-flash 호출 성공.


In [ ]:
# model2 = gemini-2.5-falsh-lite

try:
    model2 = ChatGoogleGenerativeAI(                                      # 모델 호출
        model="gemini-2.5-flash-lite",
        #temperature=0.1,
    )
    print("✅ gemini-2.5-flash-lite 호출 성공.")
except Exception as e:                                                   # 디버깅 메시지
    print(f"❌ Google GenAI 모델 초기화 실패: {e}")
    print("  -> GEMINI_API_KEY 환경 변수가 올바르게 설정되었는지 확인하세요.")      # ✅ gemini-2.5-flash-lite 호출 성공.

---


### (1) **`FewShotPromptTemplate`**

#### ➀ `gemini` 모델별 `max-token`

<br>

| 모델                    | 최대 출력 토큰 | 일반적인 용도  | 긴 글/복잡한 답변 | 매우 긴 문서/코드  |
|-----------------------|----------|----------|------------|-------------|
| Gemini 1.5 Flash      | 8,192    | 512~1024 | 4096~8192  | -           |
| Gemini 2.5 Flash      | 65,536   | 512~1024 | 4096~16384 | 32768~65536 |
| Gemini 2.5 Flash-Lite | 65,536   | 512~1024 | 4096~16384 | 32768~65536 |

<br>

***

<br>

  * `gemini-1.5-flash`
    * **최대 출력 토큰**: **8,192 토큰**
      * 권장 설정
        * 일반적인 대화, 요약, 짧은 코드: 512-1024 토큰
        * 장문의 글, 긴 코드, 복잡한 답변: 4096-8192 토큰
      * `tip!` - 상대적으로 출력 토큰이 적으므로, 응답이 중간에 잘리지 않도록 응답의 길이를 예측해서 값을 설정하는 것이 중요!

<br>

  * `gemini-2.5-flash-lite`
    * 최대 출력 토큰**: **65,536 토큰**
    * 권장 설정
        * 일반적인 대화, 요약, 짧은 코드: 512-1024 토큰
        * 장문의 글, 긴 코드, 복잡한 답변: 4096-16384 토큰
        * 매우 긴 문서 작성, 전체 책 요약, 방대한 코드 생성: 32768~65536 토큰
    * `tip!`
        * 출력 토큰 한도가 매우 높아서 대부분의 경우 넉넉하게 값을 설정할 수 있음
        * 굳이 최대치로 설정하기보다는, 필요한 만큼만 설정하여 비용을 관리하는 것을 권장

<br>

 * `결론`
    * **`일반적인 용도`** (대부분의 질문): `1024` or `2048` 토큰 정도 충분
    * **`긴 글`**, **`코드`**: **`4096 토큰 이상`으로 설정**
    * **`매우 긴 문서 요약 등 방대한 응답이 필요한 경우`**: 모델의 `최대 출력 토큰에 가깝게` 설정

In [56]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser

# 환경변수 사전 로드 필요

# --- 1. Few-shot 예시 데이터 정의 ---
# 모델 = 학습할 예시들을 파이썬 리스트 형태로 정의
# 각 예시 = 'question'과 'answer' 키를 가진 딕셔너리
examples = [
    {
        "question": "스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인
""",
    },
    {
        "question": "네이버의 창립자는 언제 태어났나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일
""",
    },
    {
        "question": "율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 율곡 이이의 어머니는 누구인가요?
중간 답변: 율곡 이이의 어머니는 신사임당입니다.
추가 질문: 신사임당은 언제 태어났나요?
중간 답변: 신사임당은 1504년에 태어났습니다.
추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
최종 답변은: 연산군
""",
    },
    {
        "question": "올드보이와 기생충의 감독이 같은 나라 출신인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 올드보이의 감독은 누구인가요?
중간 답변: 올드보이의 감독은 박찬욱입니다.
추가 질문: 박찬욱은 어느 나라 출신인가요?
중간 답변: 박찬욱은 대한민국 출신입니다.
추가 질문: 기생충의 감독은 누구인가요?
중간 답변: 기생충의 감독은 봉준호입니다.
추가 질문: 봉준호는 어느 나라 출신인가요?
중간 답변: 봉준호는 대한민국 출신입니다.
최종 답변은: 예
""",
    },
]


In [58]:
# --- 2. 예시 포맷 및 전체 프롬프트 템플릿 정의 ---
# 모델이 각 예시를 어떤 형식으로 인식할지 정의하기
example_prompt = PromptTemplate(
    input_variables=["question", "answer"],
    template="질문: {question}\n{answer}"
)

print(example_prompt.format(**examples[0]))

질문: 스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인



In [ ]:
"""

원래 코드

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)

question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"
final_prompt = prompt.format(question=question)
print(final_prompt)

"""


# Few-shot 예시들을 포함하는 최종 프롬프트 템플릿 생성
# prefix: 예시들 앞에 붙을 설명
# suffix: 예시들 뒤에 붙을 새로운 질문
# input_variables: suffix에 들어갈 변수

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="다음은 몇 가지 질문과 답변 예시입니다. 이 형식을 따라 새로운 질문에 답해주세요.\n\n",
    suffix="질문: {input}\n",
    input_variables=["input"],
)

# print(few_shot_prompt)

In [ ]:
# --- 3. LLM (대규모 언어 모델) 객체 정의 ---
# LangChain에서 Gemini 모델을 호출하는 객체 만들기
# model = 사용할 Gemini 모델명 (예: "gemini-2.5-flash-lite")
# temperature  = 0으로 설정하여 창의성을 낮추고, 예시 패턴을 엄격하게 따르도록 유도

gemini_lc = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite", temperature=0)

In [ ]:
# --- 4. LangChain Expression Language (LCEL)을 사용한 체인(Chain) 구성 ---
# 프롬프트 → LLM → 결과 파싱(parsing) 순서로 작업 연결하기
# StrOutputParser는 모델의 응답을 단순한 문자열로 변환하기

chain_fsp = few_shot_prompt | gemini_lc | StrOutputParser()

In [65]:
# --- 5. 새로운 질문으로 체인 실행 ---
# 'chain.invoke' 호출 → Few-shot 프롬프트를 모델에 전달하고 응답을 받음

new_question = "제임스 고슬링과 귀도 반 로섬 중 누가 파이썬을 만들었나요?"

response = chain_fsp.invoke({"input": new_question})

In [ ]:
# --- 6. 결과 출력 ---
print("--- Few-shot 프롬프트를 적용한 모델의 응답 ---")
print(response)

<small>

* 셀 출력 
  
    ```<markdown>
    --- Few-shot 프롬프트를 적용한 모델의 응답 ---
    이 질문에 추가 질문이 필요한가요: 예.
    추가 질문: 제임스 고슬링은 무엇을 만들었나요?
    중간 답변: 제임스 고슬링은 자바를 만들었습니다.
    추가 질문: 귀도 반 로섬은 무엇을 만들었나요?
    중간 답변: 귀도 반 로섬은 파이썬을 만들었습니다.
    최종 답변은: 귀도 반 로섬
    ```

In [ ]:
# 교안처럼 출력해보기
# FewShotPromptTemplate 객체를 그대로 사용
# 이 객체 = examples와 suffix 등의 정보를 이미 가지고 있음

# 테스트할 새로운 질문을 변수에 담기
new_question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"                        # 교안 속 질문

# `.format()` 메서드를 사용하여 최종 프롬프트 문자열을 만들기
# 'input' 변수에 new_question 값을 전달하기
final_prompt = few_shot_prompt.format(input=new_question)

# 완성된 프롬프트 문자열을 출력하기
print(final_prompt)

<small>

* 셀 출력

    ```<mardown>
    다음은 몇 가지 질문과 답변 예시입니다. 이 형식을 따라 새로운 질문에 답해주세요.



    질문: 스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?
    이 질문에 추가 질문이 필요한가요: 예.
    추가 질문: 스티브 잡스는 몇 살에 사망했나요?
    중간 답변: 스티브 잡스는 56세에 사망했습니다.
    추가 질문: 아인슈타인은 몇 살에 사망했나요?
    중간 답변: 아인슈타인은 76세에 사망했습니다.
    최종 답변은: 아인슈타인


    질문: 네이버의 창립자는 언제 태어났나요?
    이 질문에 추가 질문이 필요한가요: 예.
    추가 질문: 네이버의 창립자는 누구인가요?
    중간 답변: 네이버는 이해진에 의해 창립되었습니다.
    추가 질문: 이해진은 언제 태어났나요?
    중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
    최종 답변은: 1967년 6월 22일


    질문: 율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?
    이 질문에 추가 질문이 필요한가요: 예.
    추가 질문: 율곡 이이의 어머니는 누구인가요?
    중간 답변: 율곡 이이의 어머니는 신사임당입니다.
    추가 질문: 신사임당은 언제 태어났나요?
    중간 답변: 신사임당은 1504년에 태어났습니다.
    추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
    중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
    최종 답변은: 연산군


    질문: 올드보이와 기생충의 감독이 같은 나라 출신인가요?
    이 질문에 추가 질문이 필요한가요: 예.
    추가 질문: 올드보이의 감독은 누구인가요?
    중간 답변: 올드보이의 감독은 박찬욱입니다.
    추가 질문: 박찬욱은 어느 나라 출신인가요?
    중간 답변: 박찬욱은 대한민국 출신입니다.
    추가 질문: 기생충의 감독은 누구인가요?
    중간 답변: 기생충의 감독은 봉준호입니다.
    추가 질문: 봉준호는 어느 나라 출신인가요?
    중간 답변: 봉준호는 대한민국 출신입니다.
    최종 답변은: 예


    질문: Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?
    ```

---

In [ ]:
# 필요한 라이브러리들 불러오기
from langchain_core.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import JsonOutputParser
import os
from dotenv import load_dotenv
import json

load_dotenv()

# --- 1. Few-shot 예시 데이터 정의 ---
examples = [
    {
        "text": "새로 나온 노트북 '슈퍼북 프로'를 150만원에 구매했습니다. 성능이 좋아서 별 5개 드립니다.",
        "output": '{"상품명": "슈퍼북 프로", "가격": 1500000, "평점": 5}'
    },
    {
        "text": "이 커피 머신 '아로마 드림'은 20만원에 샀는데, 맛이 별로네요. 평점은 2점입니다.",
        "output": '{"상품명": "아로마 드림", "가격": 200000, "평점": 2}'
    },
    {
        "text": "'블루투스 이어폰 X10'을 7만원에 구매했습니다. 음질이 훌륭하여 별 4개를 주고 싶습니다.",
        "output": '{"상품명": "블루투스 이어폰 X10", "가격": 70000, "평점": 4}'
    },
]

# --- 2. 예시 포맷 및 전체 프롬프트 템플릿 정의 ---
# ChatPromptTemplate 사용
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "리뷰: {text}"),
        ("ai", "출력: {output}"),
    ]
)

# FewShotPromptTemplate 대신 FewShotChatMessagePromptTemplate 사용
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

# --- 3. LLM 객체 정의 ---
gemini_lc = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    temperature=0.8,
    response_mime_type="application/json"
)

# --- 4. 최종 프롬프트 구성 ---
# 최종 프롬프트에 system 메시지와 Few-shot 프롬프트, 그리고 새로운 질문을 포함시킵니다.
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "주어진 리뷰 문장에서 상품명, 가격, 평점을 추출하여 JSON 형식으로 출력하세요."),
    few_shot_prompt,
    ("human", "리뷰: {input}"),
])


# --- 5. 체인 구성 ---
parser = JsonOutputParser()
chain = final_prompt | gemini_lc | parser

# --- 6. 새로운 리뷰 문장으로 체인 실행 ---
new_review = "'스마트워치 A30'을 30만원에 샀는데, 디자인이 정말 마음에 듭니다. 평점은 5점입니다."
response = chain.invoke({"input": new_review})

# --- 7. 결과 출력 ---
print("--- Few-shot 프롬프트를 적용한 모델의 응답 ---")
print(response)

try:
    print("\n--- JSON 파싱 결과 ---")
    print(f"상품명: {response['상품명']}")
except Exception as e:
    print(f"⚠️ 오류 발생: {e}")

<small>

* 셀 출력 
  
  * temperature = 0 (1.3s)

    ```<markdown>
        --- Few-shot 프롬프트를 적용한 모델의 응답 ---
        {'상품명': '스마트워치 A30', '가격': 300000, '평점': 5}

        --- JSON 파싱 결과 ---
        상품명: 스마트워치 A30

    ```

<small>

***

* 올바른 조합

    ```<python>
        few_shot_prompt = FewShotChatMessagePromptTemplate(
            example_prompt=example_prompt,  # <-- ChatPromptTemplate 객체를 올바르게 처리함
            examples=examples
        )
    ```

***

* **온도(Temperature)가 출력에 영향을 주지 않는 이유**
  * `response_mime_type="application/json"`
    * 모델에게 반드시 JSON 형식으로만 응답하라는 강력한 제약 조건 = `temperature`가 높든 낮든 모델이 정해진 **문법** 을 지키도록 **강제**

  * `Few-shot 예시`
    * `FewShotChatMessagePromptTemplate`에 포함된 명확한 예시들은 모델이 특정 패턴을 따르도록 강력하게 유도
    * 이 예시들이 모델의 응답에 대한 가이드 역할 → `temperature의 영향 감소` = **특정 형식이나 패턴을 강제하는 제약 조건이 우선시**

***

***💡 tip 💡***

| 영향력 순위 | 영향 요소                            | 설명 및 이유                                                                                                                            |
|--------|----------------------------------|------------------------------------------------------------------------------------------------------------------------------------|
| 1순위    | 프롬프트 내용                          | 모델의 행동을 결정하는 가장 근본적인 요소. 명확한 지시, 작업 목표, 그리고 Few-shot 예시 등 프롬프트에 포함된 내용이 모델의 응답 방향을 90% 이상 좌우                                   |
| 2순위    | 형식 제약<br>(response_mime_type)    | 모델의 응답 형식을 강제하는 강력한 규칙. application/json과 같이 특정 형식을 지정하면, 모델은 temperature 값과 관계없이 해당 형식에 맞춰 내용을 생성해야 함 → 이 제약은 모델의 자유도를 크게 제한 |
| 3순위    | 무작위성/다양성<br>(temperature, top_p) | 모델의 창의성과 다양성을 미세하게 조절하는 요소. 프롬프트의 지시와 형식 제약 안에서 답변의 예측 가능성을 조절. 정답이 없는 창작 작업에서 중요한 역할을 하지만, 정해진 형식이 있는 작업에서는 영향력이 상대적으로 낮음   |



  * **프롬프트** = 모델이 **무엇을** 할지 알려주는 **목표**
      * **`가장 근본적인 영향`** = **모델의 출력에 가장 큰 영향을 주는 것** = **프롬프트 내용 자체입니다.**
      * **언제 중요??**
        * 모든 작업에서 중요
        * 특히 `Few-shot` 예시처럼 `명확한 지시`나 `패턴`을 줄 때 그 영향력이 극대화
        * 모델은 프롬프트에 담긴 예시를 따르도록 학습하기 때문에, 아무리 temperature를 높여도 프롬프트의 패턴을 벗어나기 어려움

  * **`response_mime_type`** = **어떻게** 해야 할지 알려주는 **가이드라인**
    * 가장 강력한 제약 = `temperature`의 영향을 **무시할 만큼 강력한 형식 제약 부여**
    * **언제 중요??**
      * `JSON`, `YAML` 등 **특정 구조**의 응답이 필요한 **엔티티 추출** → 데이터 정리 같은 작업에서 가장 중요
      * **`정해진 형식`** 에 맞춰 정보를 채워 넣는 데 집중 **>** 모델 창의성 발휘

  * **`temperature`및 `top_p`** = **그 가이드라인 안에서** 얼마나 자유롭게 할지 알려주는 **미세 조정 역할**
      * 모델의 **출력 다양성을 조절**
        * `temperature` = 무작위성, `top_p` = `확률적 선택의 범위` 제어
      * **언제 중요??**
        * 창의적인 글쓰기, 시, 스토리텔링 등 정답이 없는 `창작 작업`에서 가장 중요 → 이 값을 `높이면` 더 다양하고 예측 불가능한 답변을 얻을 수 있음

---

<small>

* next : `Example Selector`~